### Install necessary libraries

In [ ]:
!pip install bitsandbytes
!pip install transformers==4.30
!pip install -U peft
!pip install accelerate
!pip install -U trl
!pip install datasets==2.16.0
!pip install wandb

### Imports

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig,HfArgumentParser,TrainingArguments,pipeline, logging
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
import os,torch
from datasets import load_dataset
from trl import SFTTrainer
from datasets import Dataset

### Token for HuggingFace + wandb
It is recommended to use secrets in Google collab or Kaggle

In [1]:
HUGGING_FACE_HUB_TOKEN = ""
wandb=""
!huggingface-cli login --token $secret_hf

### Model initialization

In [ ]:
base_model = "mistralai/Mistral-7B-v0.1"
new_model = "copywriting"

### dataset Preparation

In [ ]:
import pandas as pd

# Load the first CSV file
df1 = pd.read_csv('/content/gdrive/My Drive/writing/MenuDataset.csv')

# Load the second CSV file
df2 = pd.read_csv('/content/gdrive/My Drive/writing/advertisingPrompt.csv')

# Load the third CSV file
df3 = pd.read_csv('/content/gdrive/My Drive/writing/social_media_prompt.csv')

# Concatenate the dataframes
appended_df = pd.concat([df1, df2, df3], ignore_index=True)

# Print the appended dataframe
appended_df.head(5)


,Unnamed: 0,Text
0,0,"<s>[INST]As a restaurant menu manager, your ro..."
1,1,"<s>[INST]As a restaurant menu manager, your ro..."
2,2,"<s>[INST]As a restaurant menu manager, your ro..."
3,3,"<s>[INST]As a restaurant menu manager, your ro..."
4,4,"<s>[INST]As a restaurant menu manager, your ro..."


In [ ]:
appended_df.shape
appended_df = appended_df.drop(columns=['Unnamed: 0'])

In [ ]:
appended_df.head(10)

,Text
0,"<s>[INST]As a restaurant menu manager, your ro..."
1,"<s>[INST]As a restaurant menu manager, your ro..."
2,"<s>[INST]As a restaurant menu manager, your ro..."
3,"<s>[INST]As a restaurant menu manager, your ro..."
4,"<s>[INST]As a restaurant menu manager, your ro..."
5,"<s>[INST]As a restaurant menu manager, your ro..."
6,"<s>[INST]As a restaurant menu manager, your ro..."
7,"<s>[INST]As a restaurant menu manager, your ro..."
8,"<s>[INST]As a restaurant menu manager, your ro..."
9,"<s>[INST]As a restaurant menu manager, your ro..."


In [ ]:
dataset = Dataset.from_pandas(appended_df)
dataset

Dataset({
    features: ['Text'],
    num_rows: 4826
})

# Modelling

In [ ]:
from accelerate import Accelerator

# Load base model(Mistral 7B)
bnb_config = BitsAndBytesConfig(
    load_in_4bit= True,
    bnb_4bit_quant_type= "nf4",
    bnb_4bit_compute_dtype= torch.bfloat16,
    bnb_4bit_use_double_quant= False,
)
model = AutoModelForCausalLM.from_pretrained(
        base_model,
        quantization_config=bnb_config,
        torch_dtype=torch.bfloat16,
        device_map={"": 0},
        offload_folder="offload/",
        trust_remote_code=True,
)
model.config.use_cache = False # silence the warnings. Please re-enable for inference!
model.config.pretraining_tp = 1
model.gradient_checkpointing_enable()

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.padding_side = 'right'
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_eos_token = True
tokenizer.add_bos_token, tokenizer.add_eos_token

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

(True, True)

In [ ]:
#Adding the adapters in the layers
model = prepare_model_for_kbit_training(model)
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj","gate_proj"]
)
model = get_peft_model(model, peft_config)

In [ ]:
#Hyperparamter
training_arguments = TrainingArguments(
    output_dir="copywriting/",
    num_train_epochs=1,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=25,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="wandb"
)


In [ ]:
# Setting sft parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    max_seq_length= None,
    dataset_text_field="Text",
    tokenizer=tokenizer,
    args=training_arguments,
    packing= False,
)

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:245: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/4826 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


### Training

In [ ]:
trainer.train()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
25,1.180900
50,0.551100
75,0.833000
100,0.497000
125,0.770000
150,0.479200
175,0.686700
200,0.472500
225,0.684200
250,0.426900


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.

TrainOutput(global_step=1207, training_loss=0.5178593722077175, metrics={'train_runtime': 8680.01, 'train_samples_per_second': 0.556, 'train_steps_per_second': 0.139, 'total_flos': 6.775587544439194e+16, 'train_loss': 0.5178593722077175, 'epoch': 1.0})

### Saving the adaopter layer

In [ ]:
# Save the fine-tuned model
trainer.model.save_pretrained(new_model)
#wandb.finish()
model.config.use_cache = True
model.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32000, 4096)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralSdpaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=64, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=64, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): lora.Linear4bit(
                (base_layer): 

###. Push to Hub

In [ ]:

try:
    trainer.model.push_to_hub(new_model, use_temp_dir=False)
except:
    print("An exception occurred")